In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import tweepy as tw
from tweepy import OAuthHandler
import nltk
from nltk.corpus import stopwords
import re
import networkx
import time
import datetime
from datetime import datetime, timedelta
import json
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from tkinter import font

import warnings
warnings.filterwarnings("ignore")

nltk.download('stopwords')

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

consumer_key = 'xx0RCPAc9HTerVYq7t1ppOfjG'
consumer_secret = 'u0JAi7wAQpvKmQDvyiW5oe7HwIXw2ijul6iwfJWOjg1totiuUX'
access_token = '962871151616778240-mhZsHGoDaXeoeAaG6VydUJEiGw5AnCo'
access_token_secret = 'fXy4nVEHDlPb7aYNEIzSpfW5vudI2SzGv6ueQBkKRTRqf'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

root = Tk()
root.title("Twitter Hashtag Visualization Tool.")

C13 = font.Font(family="Arial", size=13, weight="bold")

label1 = ttk.Label(root, text = "Type The Hashtag That You Would Like To Search Twitter For Here:", font = C13)
label2 = ttk.Label(root, text = "Remember To Provide Your Input As 1 Word/Phrase.", font = C13)
label3 = ttk.Label(root, text = "Example Inputs: (1) climatechange or (2) fakenews", font = C13)

E1 = ttk.Entry(root)

def getE1():
    return E1.get()
def mainFunction():
    try:
        getE1()
        input = getE1()
        search_words = "#" + str(input)
        past_date_before_4_days = datetime.now() - \
                                  timedelta(days = 4)
        date_since = str(past_date_before_4_days)[0:10]
        tweets = tw.Cursor(api.search, q=search_words, lang="en", since = date_since).items(1000)
        all_tweets = [tweet.text for tweet in tweets]
        def remove_url(txt):
            return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
        all_tweets_no_urls = [remove_url(tweet) for tweet in all_tweets]
        words_in_tweet = [tweet.lower().split() for tweet in all_tweets_no_urls]
        stop_words = set(stopwords.words('english'))
        for all_words in words_in_tweet:
            for word in all_words:
                tweets_nsw = [[word for word in tweet_words if not word in stop_words] for tweet_words in words_in_tweet]
        all_words_nsw = list(itertools.chain(*tweets_nsw))
        counts_nsw = collections.Counter(all_words_nsw)
        clean_tweets_nsw = pd.DataFrame(counts_nsw.most_common(15), columns=['words', 'count'])
        fig, ax = plt.subplots(figsize=(8, 8))
        clean_tweets_nsw.sort_values(by='count').plot.barh(x='words', y='count', ax=ax, color="purple")
        ax.set_title("Common Words Found In Such Tweets!")
        plt.show()
        popup1 = messagebox.showinfo('Result', "A visualization for this specific hashtag is shown below." + "\nThank you for using this tool!")
    except tw.error.TweepError as e:
        print(e.reason)
        popup2 = messagebox.showerror('Result', 'Something seems to have gone wrong with the program! Please try again later.' + "\nThank you for using this tool!")
    except StopIteration:
        popup3 = messagebox.showerror('Result', 'Something seems to have gone wrong with the program! Please try again later.' + "\nThank you for using this tool!")

def ClearEntry():
    E1.delete(first = 0, last = 'end')

button1 = ttk.Button(root, text = "SUBMIT", command = mainFunction)
button2 = ttk.Button(root, text = "EXIT PAGE", width = 16, command = root.destroy)
button3 = ttk.Button(root, text = "CLEAR ENTRIES", width = 18, command = ClearEntry)

label1.pack()

E1.pack()

label2.pack()

label3.pack()

button1.pack(side = TOP)

button2.pack(side = TOP)

button3.pack(side = TOP)

root.mainloop()